# CODE FOR LANDMARK ENSEMBLE
## Without bagging ver.

# 1. Load Data & simple Pre-processing & Setting

In [1]:
from landmark_ensemble.functions import * 
import dill
from datetime import datetime

In [2]:
####################################################################################################################################

# settings 
dir = "/Users/pio/Google 드라이브/data/"
file_name = "pbc2.csv"
data = pd.read_csv(dir + file_name)

# drop status1 - competing risks setting
data = data.drop(axis=1, columns =['status'])


# ID, Time, Event, Measure Time column names
ID_col = 'id'; T_col ='years'; E_col ='status2'; measure_T_col = 'year'

# categorical variables
nominal_col = ['drug','sex', 'ascites', 'hepatomegaly','spiders', 'edema']
ordinal_col = ['histologic']

# continuous variables
cont_col = list(set(data.columns) - set(nominal_col) - set(ordinal_col) - set([ID_col, T_col, E_col, measure_T_col]))

# window - 5 year prediction 
window = 5

# S : landmark time points - 0, 0.5, 1, ..., 10
S = np.linspace(0,10,21)
v_years = S+window

# Number of bins when discritizing 
## !!!(Actually, k_bin - 1 bins are produced)!!!
k_bin = 5

# minimal bin_size
minimal_bin_size = window / (k_bin-1)

# 

# for continous variables, 
## scaling -> min-max scaling &
## imputation -> fill na's : median for continous
for col in cont_col : 
    data[col] = data[col].fillna(data[col].median())
    data[col] = (data[col] - min(data[col])) / (max(data[col]) - min(data[col]))

# one-hot encoding for categorical variables
data = pd.get_dummies(data, columns = nominal_col, drop_first=True)


####################################################################################################################################
# settings2

# proportion of train set
p_train = 0.7
k_kfold = 3

In [4]:
## model specifics of level 0 models
cox_params = {'penalizer':np.exp(np.linspace(-5,1,5)),'l1_ratio':[0,0.25,0.5,0.75,1]}
# 5*5 *2 = 50
model_specifics_cont = pd.DataFrame({'model_name' : ['cox_str', 'cox_no_str'], 
                                'model_instance':[CoxPHFitter(),CoxPHFitter()], 
                                'hyperparams':[cox_params,cox_params], 
                                'type':['cox_str','cox_no_str']})

LR_params = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
    'penalty': ['l1', 'l2'],
    'solver': ['saga']
} # 7 * 2 * 1 = 14
RF_params = {'n_estimators':[50,100,300,500],'max_depth':[1,3,5]} # 4*3 = 12
GB_params = {'n_estimators':[50,100,300,500],'max_depth':[1,3,5]} # 4*3 = 12
MLP_params = {'hidden_layer_sizes':[1,2], 'activation' : ['logistic', 'relu'], 'max_iter' : [1000], 'early_stopping' : [True], 'learning_rate' : ['adaptive']}
# 2*2 = 4
KNN_params = {'n_neighbors':[1,5,10], 'weights':['uniform', 'distance']} 
# 3*2
NGB_params = {'var_smoothing':[1e-5, 1e-9, 1e-1]}
# 3
ADA_params = {'n_estimators':[50, 100, 300, 500], 'max_depth':[1,3,5]}
# 4*10*3 = 36

model_specifics_disc = pd.DataFrame({'model_name' : ['LR','RF','GB','MLP','KNN','NGB','ADA'], 
                                'model_instance':[LogisticRegression(max_iter=10000),RandomForestClassifier(),GradientBoostingClassifier(),MLPClassifier(),KNeighborsClassifier(),GaussianNB(), AdaBoostClassifier()], 
                                'hyperparams':[LR_params, RF_params, GB_params,MLP_params, KNN_params,NGB_params, ADA_params], 
                                'type':['lr','rf','gb','mlp','knn','ngb','ada']})


model_specifics = pd.concat([model_specifics_cont,model_specifics_disc],axis=0).reset_index(drop=True)
model_specifics

,model_name,model_instance,hyperparams,type
0,cox_str,<lifelines.CoxPHFitter>,"{'penalizer': [0.006737946999085467, 0.0301973...",cox_str
1,cox_no_str,<lifelines.CoxPHFitter>,"{'penalizer': [0.006737946999085467, 0.0301973...",cox_no_str
2,LR,LogisticRegression(max_iter=10000),"{'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000], 'p...",lr
3,RF,RandomForestClassifier(),"{'n_estimators': [50, 100, 300, 500], 'max_dep...",rf
4,GB,GradientBoostingClassifier(),"{'n_estimators': [50, 100, 300, 500], 'max_dep...",gb
5,MLP,MLPClassifier(),"{'hidden_layer_sizes': [1, 2], 'activation': [...",mlp
6,KNN,KNeighborsClassifier(),"{'n_neighbors': [1, 5, 10], 'weights': ['unifo...",knn
7,NGB,GaussianNB(),"{'var_smoothing': [1e-05, 1e-09, 0.1]}",ngb
8,ADA,AdaBoostClassifier(),"{'n_estimators': [50, 100, 300, 500], 'max_dep...",ada


# 2. Landmarking

In [5]:
data_lm_cont = landmarker_cont(data=data, ID_col = ID_col, T_col = T_col, E_col = E_col, 
                window = window, S= S, measure_T_col = measure_T_col)

data_lm_disc = landmarker_disc(data=data_lm_cont,ID_col = ID_col, T_col = T_col, E_col = E_col, 
                window = window, S= S, measure_T_col = measure_T_col, k_bin = k_bin, train=True)



# 3. Loop

In [6]:
dir_save = '/Users/pio/Google 드라이브/github/survival ensemble/experiment/'
n_rep = 50

In [10]:
for seed_num in range(n_rep) : 
    current_time = datetime.now().strftime('%H:%M ')
    print('##############################')
    print('Current Seed =', seed_num)
    print('Current Time =', current_time)

    ####### <TRAIN-TEST SPLIT>
    # Split IDs into train set and test set
    train_id, test_id = id_train_test_split(id_list = data[ID_col], seed_number = seed_num, p=0.7)

    # Train, test set from original form
    train = data[data[ID_col].isin(train_id)].reset_index(drop=True)
    test = data[data[ID_col].isin(test_id)].reset_index(drop=True)

    # Train, test set for continous landmarking algorithms
    train_lm_cont = data_lm_cont[data_lm_cont[ID_col].isin(train_id)].reset_index(drop=True)
    test_lm_cont = data_lm_cont[data_lm_cont[ID_col].isin(test_id)].reset_index(drop=True)

    # Train, test set for discrete landmarking algorithms
    train_lm_disc = data_lm_disc[data_lm_disc[ID_col].isin(train_id)].reset_index(drop=True)
    test_lm_disc = data_lm_disc[data_lm_disc[ID_col].isin(test_id)].reset_index(drop=True)

#    print(np.all(np.unique(train_lm_cont.id) == np.unique(train_lm_disc.id)))
#    print(np.all(np.unique(test_lm_cont.id) == np.unique(test_lm_disc.id)))

    ####### <TRAINING PART>
    # 1. Generating dataset for training meta model part 
    
    kfold = id_kfold(id_list=train_id, n_split=k_kfold,seed_number=seed_num)
    stacked_trn = []
    print('Generating dataset for training meta model')
    for i in range(k_kfold) : 
        print('fold : ' + str(i))
        k_fold_trn_id, k_fold_val_id = next(kfold)

        k_fold_trn_lm_cont = train_lm_cont[train_lm_cont[ID_col].isin(k_fold_trn_id)].copy()
        k_fold_trn_lm_disc = train_lm_disc[train_lm_disc[ID_col].isin(k_fold_trn_id)].copy()

        k_fold_val_lm_cont = train_lm_cont[train_lm_cont[ID_col].isin(k_fold_val_id)].copy()
        k_fold_val_lm_disc = train_lm_disc[train_lm_disc[ID_col].isin(k_fold_val_id)].copy()

        # fit all baseline models        
        stack_fit = stacker(model_specifics = model_specifics, 
                            ID = ID_col, T = T_col, E = E_col, S = S, window = window, k_bin = k_bin)
        stack_fit.fit(data_cont= k_fold_trn_lm_cont , data_disc = k_fold_trn_lm_disc) 

        # stack them for training meta model
        stacked_trn.append(stack_fit.predict(k_fold_val_lm_cont, k_fold_val_lm_disc))

    # ID_col, LM, T_col, E_col validation 순서에 맞게 모으기
    info = pd.concat([train_lm_cont[train_lm_cont[ID_col].isin(kfold.validation_fold_id[i])][[ID_col, 'LM', T_col, E_col]].reset_index(drop=True) for i in range(len(stacked_trn))], ignore_index=True)
    # kfold validation 예측 결과 모으기
    pred = b = pd.concat([pd.DataFrame(stacked_trn[i]) for i in range(len(stacked_trn))], ignore_index=True)
    new_data = pd.concat([info,pred], axis=1)
    # new_data['surv_status'] = abs(new_data[E_col]-1)
    ######
    # 2. Training Part : 
    # 2-1. (Re-)train baseline models on whole dataset  
    print('Re-train baseline models')
    stack_fit = stacker(model_specifics = model_specifics, 
                            ID = ID_col, T = T_col, E = E_col, S = S, window = window, k_bin = k_bin)

    stack_fit.fit(data_cont=train_lm_cont.copy() , data_disc = train_lm_disc.copy()) 

    ##### 
    # 2-2-1. calculating ipcw weights( for supplying weights to meta models)
    ipcw_calc = ipcw_fitter(S= S, window =window)
    ipcw_calc.fit(data= new_data, T = T_col, E = E_col)

    # 2-2-2. Meta model Training 및 3. Prediction으로 연결.  
    
    ####### <SAVE PART>
    # 
    with open(dir_save +'seed_'+str(seed_num)+'_'+'new_data.pkl', 'wb') as f : 
        dill.dump(new_data,f)

    with open(dir_save +'seed_'+str(seed_num)+'_'+'stack_fit.pkl', 'wb') as f : 
        dill.dump(stack_fit,f)

    with open(dir_save +'seed_'+str(seed_num)+'_'+'ipcw_fit.pkl', 'wb') as f : 
        dill.dump(ipcw_calc,f)
    
    train_test_id_dict = {'train_id':train_id, 'test_id':test_id}
    with open(dir_save +'seed_'+str(seed_num)+'_'+'train_test_id_dict.pkl', 'wb') as f : 
        dill.dump(train_test_id_dict,f)




##############################
Current Seed = 30
Current Time = 01:06 
Generating dataset for training meta model
fold : 0
fold : 1
fold : 2
Re-train baseline models
##############################
Current Seed = 31
Current Time = 01:25 
Generating dataset for training meta model
fold : 0
fold : 1
fold : 2
Re-train baseline models
##############################
Current Seed = 32
Current Time = 01:42 
Generating dataset for training meta model
fold : 0
fold : 1
fold : 2
Re-train baseline models
##############################
Current Seed = 33
Current Time = 02:01 
Generating dataset for training meta model
fold : 0
fold : 1
fold : 2
Re-train baseline models
##############################
Current Seed = 34
Current Time = 02:20 
Generating dataset for training meta model
fold : 0
fold : 1
fold : 2
Re-train baseline models
##############################
Current Seed = 35
Current Time = 02:38 
Generating dataset for training meta model
fold : 0
fold : 1
fold : 2
Re-train baseline models
####

# 4. Load models  & data again and fit meta model

### 메트릭 계산 안 되는 seed : 8, 12, 27

In [98]:
dir_save = '/Users/pio/Google 드라이브/github/survival ensemble/experiment/'

for seed_num in range(28,50) : 
    print('########')
    print(seed_num)
    # loading
    with open(dir_save +'seed_'+str(seed_num)+'_'+'new_data.pkl', 'rb') as f : 
        new_data = dill.load(f)

    with open(dir_save +'seed_'+str(seed_num)+'_'+'stack_fit.pkl', 'rb') as f : 
        stack_fit = dill.load(f)

    with open(dir_save +'seed_'+str(seed_num)+'_'+'ipcw_fit.pkl', 'rb') as f : 
        ipcw_calc = dill.load(f)

    with open(dir_save +'seed_'+str(seed_num)+'_'+'train_test_id_dict.pkl', 'rb') as f : 
        train_test_id_dict = dill.load(f)


    new_x = new_data.drop([ID_col, 'LM', T_col, E_col],axis=1)
    new_y = abs(new_data[E_col]-1)  # 생존확률의 결합이므로 라벨을 뒤집어줘야 함.
    new_w = ipcw_calc.predict(new_data) # 

    print(new_x.shape)
    print(new_y.shape)
    print(new_w.shape)

    train_lm_cont = data_lm_cont[data_lm_cont[ID_col].isin(train_test_id_dict['train_id'])].reset_index(drop=True)
    train_lm_disc = data_lm_disc[data_lm_disc[ID_col].isin(train_test_id_dict['train_id'])].reset_index(drop=True)

    test_lm_cont = data_lm_cont[data_lm_cont[ID_col].isin(train_test_id_dict['test_id'])].reset_index(drop=True)
    test_lm_disc = data_lm_disc[data_lm_disc[ID_col].isin(train_test_id_dict['test_id'])].reset_index(drop=True)

    # fit meta model
    nnls = nnls_constraint()
    nnls.fit(x = new_x, 
             y = new_y,
             w = new_w)


    hill = hillclimb()
    hill.fit(x = new_x, 
             y = new_y,
             w = new_w)


    ipcw_rf = RandomForestClassifier()
    ipcw_rf.fit(X = new_x, 
                y = new_y, 
                sample_weight = new_w) 

    # predict
    test_new_x = stack_fit.predict(data_cont = test_lm_cont, data_disc = test_lm_disc) 

    nnls_pred = nnls.predict(test_new_x)
    hill_pred = hill.predict(test_new_x)
    ipcw_rf_pred = ipcw_rf.predict_proba(test_new_x)[:,1]
    
    # scores
    ## Brier
    
    test_ipcw_calc = ipcw_fitter(S= S, window =window)
    test_ipcw_calc.fit(data= test_lm_cont, T = T_col, E = E_col)
    test_ipcw_pred = test_ipcw_calc.predict(data= test_lm_cont)

    # i for model, j for landmarked time
    brier_score_list = []
    for i in range(test_new_x.shape[1]) : 
        temp = []
        for j in S : 
            value = brier_score_loss(y_true = abs(test_lm_cont[E_col]-1)[test_lm_cont['LM'] == j], 
                             y_prob = pd.DataFrame(test_new_x)[test_lm_cont['LM'] == j][i], 
                             sample_weight= test_ipcw_pred[test_lm_cont['LM'] == j])
            temp.append(value)        
        brier_score_list.append(temp)

    result_brier_base = pd.DataFrame(brier_score_list)
    
    brier_nnls = [] ; brier_hill = [] ; brier_rf = []
    for j in S : 
    #    brier_cox.append(brier_score_loss(y_true = abs(test_lm_cont[E_col]-1)[test_lm_cont['LM'] == j], 
    #                         y_prob = pd.DataFrame(test_new_x)[test_lm_cont['LM'] == j][75], 
    #                         sample_weight= test_ipcw_pred[test_lm_cont['LM'] == j]))
        brier_nnls.append(brier_score_loss(y_true = abs(test_lm_cont[E_col]-1)[test_lm_cont['LM'] == j], 
                         y_prob = nnls_pred[test_lm_cont['LM'] == j], 
                         sample_weight= test_ipcw_pred[test_lm_cont['LM'] == j]))        
        brier_hill.append(brier_score_loss(y_true = abs(test_lm_cont[E_col]-1)[test_lm_cont['LM'] == j], 
                         y_prob = hill_pred[test_lm_cont['LM'] == j], 
                         sample_weight= test_ipcw_pred[test_lm_cont['LM'] == j]))
        brier_rf.append(brier_score_loss(y_true = abs(test_lm_cont[E_col]-1)[test_lm_cont['LM'] == j], 
                         y_prob = ipcw_rf_pred[test_lm_cont['LM'] == j], 
                         sample_weight= test_ipcw_pred[test_lm_cont['LM'] == j]))


    result_brier_meta = pd.DataFrame({'nnls': brier_nnls, 'hill':brier_hill, 'rf':brier_rf})
    result_brier_total = pd.concat([pd.DataFrame(np.array(result_brier_base).T), result_brier_meta],axis=1)
         
    with open(dir_save +'seed_'+str(seed_num)+'_'+'brier_score.pkl', 'wb') as f : 
        dill.dump(result_brier_total,f)
        
    ## C-index
    # i for model, j for landmarked time
    c_index_list = []
    for i in range(test_new_x.shape[1]) : 
        temp = []
        for j in S : 
            c_index_value = concordance_index(event_times = test_lm_cont[test_lm_cont['LM'] == j][T_col], 
                                              predicted_scores = pd.DataFrame(test_new_x)[test_lm_cont['LM'] == j][i],
                                              event_observed = test_lm_cont[test_lm_cont['LM'] == j][E_col])
            temp.append(c_index_value)        
        c_index_list.append(temp)

    result_c_base = pd.DataFrame(c_index_list)
    
    
    c_index_nnls = []; c_index_hill = []; c_index_rf = [] 
    for j in S : 
        c_index_nnls.append(concordance_index(event_times = test_lm_cont[test_lm_cont['LM'] == j][T_col], 
                      predicted_scores = nnls_pred[test_lm_cont['LM'] == j],
                      event_observed = test_lm_cont[test_lm_cont['LM'] == j][E_col])

    )        
        c_index_hill.append(concordance_index(event_times = test_lm_cont[test_lm_cont['LM'] == j][T_col], 
                      predicted_scores = hill_pred[test_lm_cont['LM'] == j],
                      event_observed = test_lm_cont[test_lm_cont['LM'] == j][E_col])

    )
        c_index_rf.append(concordance_index(event_times = test_lm_cont[test_lm_cont['LM'] == j][T_col], 
                      predicted_scores = ipcw_rf_pred[test_lm_cont['LM'] == j],
                      event_observed = test_lm_cont[test_lm_cont['LM'] == j][E_col])

    )

    result_c_meta = pd.DataFrame({'nnls': c_index_nnls, 'hill':c_index_hill, 'rf':c_index_rf})
    result_c_total = pd.concat([pd.DataFrame(np.array(result_c_base).T),result_c_meta], axis=1)
    with open(dir_save +'seed_'+str(seed_num)+'_'+'c_index_score.pkl', 'wb') as f : 
        dill.dump(result_c_total,f)

        
    print(sum(np.sum(np.isnan(result_brier_total))))
    print(sum(np.sum(np.isnan(result_c_total))))


########
28
(2847, 113)
(2847,)
(2847,)
0
0
########
29
(2729, 113)
(2729,)
(2729,)
0
0
########
30
(2764, 113)
(2764,)
(2764,)
0
0
########
31
(2798, 113)
(2798,)
(2798,)
0
0
########
32
(2806, 113)
(2806,)
(2806,)
0
0
########
33
(2836, 113)
(2836,)
(2836,)
0
0
########
34
(2835, 113)
(2835,)
(2835,)
0
0
########
35
(2855, 113)
(2855,)
(2855,)
0
0
########
36
(2737, 113)
(2737,)
(2737,)
0
0
########
37
(2747, 113)
(2747,)
(2747,)
0
0
########
38
(2755, 113)
(2755,)
(2755,)
0
0
########
39
(2681, 113)
(2681,)
(2681,)
0
0
########
40
(2834, 113)
(2834,)
(2834,)
0
0
########
41
(2853, 113)
(2853,)
(2853,)
0
0
########
42
(2771, 113)
(2771,)
(2771,)
0
0
########
43
(2820, 113)
(2820,)
(2820,)
0
0
########
44
(2777, 113)
(2777,)
(2777,)
0
0
########
45
(2836, 113)
(2836,)
(2836,)
0
0
########
46
(2772, 113)
(2772,)
(2772,)
0
0
########
47
(2763, 113)
(2763,)
(2763,)
0
0
########
48
(2788, 113)
(2788,)
(2788,)
0
0
########
49
(2834, 113)
(2834,)
(2834,)
0
0
